In [2]:
import torch
from torchvision import datasets,transforms as T,models
from torch.utils.data import DataLoader
import numpy as np 
from collections import OrderedDict
from torch import optim,nn
import matplotlib.pyplot as plt 
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import seaborn as sns

In [3]:
train_dir = 'data/train'
valid_dir = 'data/val'
test_dir = 'data/test'

In [4]:
train_transform = T.Compose([
    T.Resize((320,320)),
    T.RandomRotation(0,359),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transform = T.Compose([
    
    T.Resize((320,320)),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    
])

test_transform = valid_transform

In [5]:
trainset = datasets.ImageFolder(train_dir,transform = train_transform)

validset = datasets.ImageFolder(valid_dir,transform = valid_transform)

testset = datasets.ImageFolder(test_dir,transform = test_transform)

In [6]:
Trainloader = DataLoader(trainset, batch_size = 64, shuffle = True)
Validloader = DataLoader(validset, batch_size = 64, shuffle = True)
Testloader = DataLoader(testset, batch_size = 64, shuffle = True)

In [7]:
model = models.inception_v3(pretrained=True)

In [8]:
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict

fc = nn.Sequential(OrderedDict([
    ('fc1',nn.Linear(2048,1024)),
    ('relu',nn.ReLU()),
    ('dropout',nn.Dropout(0.5)),
    ('fc2',nn.Linear(1024,500)),
    ('relu',nn.ReLU()),
    ('dropout',nn.Dropout(0.5)),
    ('fc3',nn.Linear(500,2)),
    ('output',nn.LogSoftmax(dim = 1))
]))

model.fc = fc
model.cuda()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [9]:
model.load_state_dict(torch.load('PneumoniaModel.pt'))

<All keys matched successfully>

In [10]:
criterion = nn.NLLLoss()

In [11]:
train_acc = 0 
valid_acc = 0 
test_acc = 0 
test_loss = 0 
train_loss = 0
val_loss = 0
true_label = []
predict_label = []

for images,labels in Trainloader:
  images = images.cuda()
  labels = labels.cuda()
  logps,aux = model(images)
  loss = criterion(logps,labels)
  train_loss += loss.item()*images.size(0)
  ps = torch.exp(logps)
  top_p, top_class = ps.topk(1, dim = 1)
  T_equals = top_class == labels.view(*top_class.shape)
  train_acc += torch.mean(T_equals.type(torch.FloatTensor))

for images,labels in Validloader:
  images = images.cuda()
  labels = labels.cuda()
  logps,aux = model(images)
  loss = criterion(logps,labels)
  val_loss += loss.item()*images.size(0)
  ps = torch.exp(logps)
  top_p,top_class = ps.topk(1, dim = 1)
  V_equals = top_class == labels.view(*top_class.shape)
  valid_acc += torch.mean(V_equals.type(torch.FloatTensor))

for images,labels in Testloader:
  images = images.cuda()
  labels = labels.cuda()
  l = labels.type(torch.FloatTensor)
  true_label.append(l)
  logps,aux = model(images)
  loss = criterion(logps,labels)
  l1 = loss.type(torch.FloatTensor)
  predict_label.append(l1)
  test_loss += loss.item()*images.size(0)
  ps = torch.exp(logps)
  top_p,top_class = ps.topk(1, dim = 1)
  t_equals = top_class == labels.view(*top_class.shape)
  test_acc += torch.mean(t_equals.type(torch.FloatTensor))

print("Overall Training Accuracy : {}\n".format(train_acc/len(Trainloader)))
print("Overall Validation Accuracy : {}\n".format(valid_acc/len(Validloader)))
print("Overall Test Accuracy : {}\n".format(test_acc/len(Testloader)))
print("Overall Train Loss : {}\n".format(train_loss/len(Trainloader.dataset)))
print("Overall Valid Loss : {}\n".format(val_loss/len(Validloader.dataset)))
print("Overall Test Loss : {}\n".format(test_loss/len(Testloader.dataset)))

Overall Training Accuracy : 0.8567073345184326

Overall Validation Accuracy : 0.6875

Overall Test Accuracy : 0.7723958492279053

Overall Train Loss : 0.2812514480637626

Overall Valid Loss : 0.6718666553497314

Overall Test Loss : 0.543766704125282

